In [1]:
"""
Purpose: To show that the caveclient vdi 
is working and can fetch meshes and synapses
"""

'\nPurpose: To show that the caveclient vdi \nis working and can fetch meshes and synapses\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from mesh_tools import trimesh_utils as tu
from datasci_tools import ipyvolume_utils as ipvu
from neurd import neuron_visualizations as nviz

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/local/lib/python3.8/dist-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


In [4]:
from neurd import cave_client_utils as ccu

In [5]:
from neurd.vdi_microns_cave import volume_data_interface as vdi

In [6]:
vdi.voxel_to_nm_scaling

array([ 4.,  4., 40.])

In [7]:
vdi.client.segment_ids_with_nucleus()

Table Owner Notice on nucleus_ref_neuron_svm: Please cite https://doi.org/10.1101/2022.07.20.499976 when using this table.


array([                 0, 864691131906133483, 864691132144578807, ...,
       864691137198243393, 864691137198252609, 864691137198255681])

# Downloading mesh and synapses

In [8]:
seg_id = segment_id= 864691137198252609

In [9]:
mesh = vdi.fetch_segment_id_mesh(seg_id)
mesh

<trimesh.Trimesh(vertices.shape=(2523209, 3), faces.shape=(5063649, 3))>

In [10]:
syn_df = vdi.segment_id_to_synapse_dict(seg_id)
syn_df

,segment_id,segment_id_secondary,synapse_id,prepost,synapse_x,synapse_y,synapse_z,synapse_size
0,864691137198252609,864691135902080478,215360645,pre,842440,384256,1013760,552960
1,864691137198252609,864691135758973006,173939647,pre,747336,487324,1027080,360960
2,864691137198252609,864691135319320488,203064053,pre,804224,793304,1071280,8524800
3,864691137198252609,864691136272989374,241323899,pre,909264,330448,959760,1492480
4,864691137198252609,864691133114428938,195794757,pre,797708,612376,970080,1085440
...,...,...,...,...,...,...,...,...
4234,864691137198252609,864691136237813820,198757478,post,805064,509808,893800,1313280
4235,864691137198252609,864691135754357197,205749107,post,818344,460440,1005760,737280
4236,864691137198252609,864691135394307317,198655508,post,807856,493328,971800,3816960
4237,864691137198252609,864691135700772731,191447790,post,795392,496568,927840,1704960


# Plotting the results

In [11]:
from datasci_tools import ipyvolume_utils as ipvu

"""
Purpose: Plot the mesh and the synapses

Pseudocode: 
1) convert the 
"""
syn_coords = syn_df[[f"synapse_{k}" for k in ["x","y","z"]]].to_numpy()

ipvu.plot_objects(
    mesh,
    scatters=[syn_coords],
)

/usr/local/lib/python3.8/dist-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

# Step 0: Setting Container to Save Results

In [12]:
from datasci_tools import pipeline

products = pipeline.PipelineProducts()
products

# Step 1: Decimation

In [13]:
"""
NOTE: an already decimated mesh is used for the example
in order to speed up time, that's why the mesh_dec
object will be overwritten with original mesh object
"""

"\nNOTE: an already decimated mesh is used for the example\nin order to speed up time, that's why the mesh_dec\nobject will be overwritten with original mesh object\n"

In [14]:
decimation_parameters = dict(
    decimation_ratio =0.25,
)

In [15]:
mesh_decimated = tu.decimate(
    mesh,
    **decimation_parameters
)

xvfb-run -n 6933 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/VDI_microns/temp/neuron_56751.off -o /NEURD/Applications/Tutorials/VDI_microns/temp/neuron_56751_decimated.off -s /NEURD/Applications/Tutorials/VDI_microns/temp/decimation_meshlab_2543293.mls


In [16]:
# saving any parameters used or output products for later use
products.set_stage_attrs(
    stage = "decimation",
    attr_dict = dict(
        decimation_parameters = decimation_parameters,
        segment_id = segment_id,
    ),
    
)

print(products)

{
  'decimation':
  {
    'decimation_parameters':
    {
      'decimation_ratio':0.25,
    },
    'segment_id':864691137198252609,
  },
}



# Step 2: Soma Identification

In [17]:
from neurd import soma_extraction_utils as sm
soma_extraction_parameters = dict()

In [18]:
# could inspect the parameters of a certain category
from neurd import parameter_utils as paru
paru.category_param_from_module(
        module = sm,
        category = "glia",
)

{'glia_volume_threshold_in_um_global': 2500,
 'glia_n_faces_threshold_global': 400000,
 'glia_n_faces_min_global': 100000}

In [19]:
from neurd import neuron_visualizations as nviz

verbose = True

soma_products = sm.soma_indentification(
    mesh_decimated,
    verbose=verbose,
    **soma_extraction_parameters
)

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 5000.0 
large_mesh_threshold_inner = 3250.0 
soma_size_threshold = 562.5 
soma_size_threshold_max = 75000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
max_mesh_sized_filtered_away = 22500.0
xvfb-run -n 8634 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/VDI_microns/temp/neuron_98186.off -o /NEURD/Applications/Tutorials/VDI_microns/temp/neuron_98186_remove_interior.off -s /NEURD/Applications/Tutorials/VDI_microns/temp/remove_interior_220887.mls
removed temporary input file: /NEURD/Applications/Tutorials/VDI_microns/temp/neuron_98186.off
removed temporary output file: /NEURD/Applications/Tutorials/VDI_microns/temp/neuron_98186_remove_interior.off
/NEURD/Applications/Tutorials/VDI_microns/temp/remove_interior_220887.mls is being deleted....
There were 13 total interior meshes
Pieces satisfying glia requirements (volume) (x >= 2500000000000): 0
Pieces satisfying nuclie r

  0%|          | 0/9 [00:00<?, ?it/s]

      ------ Found 1 viable somas: [0.86286]
Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 3027 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/VDI_microns/Poisson_temp/neuron_607448.off -o /NEURD/Applications/Tutorials/VDI_microns/Poisson_temp/neuron_607448_poisson.off -s /NEURD/Applications/Tutorials/VDI_microns/Poisson_temp/poisson_294388.mls
removed temporary input file: /NEURD/Applications/Tutorials/VDI_microns/Poisson_temp/neuron_607448.off
removed temporary output file: /NEURD/Applications/Tutorials/VDI_microns/Poisson_temp/neuron_607448_poisson.off
mesh.is_watertight = True
/NEURD/Applications/Tutorials/VDI_microns/Poisson_temp/poisson_294388.mls is being deleted....
Inside sphere validater: ratio_val = 2.075305542328065



 Total time for run = 225.38344073295593
Before Filtering the number of somas found = 1

---Performing Soma Mesh Backtracking to original mesh for poisson soma 0
backtrack_soma_size_thre

  0%|          | 0/10 [00:00<?, ?it/s]

Largest hole before segmentation = 58008.09339792085, after = 272571.3444482195,

ratio = 4.698850254885108, difference = 214563.25105029868
filtered_soma_list_components = [<trimesh.Trimesh(vertices.shape=(16645, 3), faces.shape=(33075, 3))>]
Not need to do a second pass because already found a soma


In [20]:
sm.plot_soma_products(
    mesh_decimated,
    soma_products = soma_products,
    verbose = True
)

# of somas = 1
# of glia = 0
# of nuclei = 9


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

In [21]:
soma_products.soma_extraction_parameters = soma_extraction_parameters
soma_products

In [22]:
products.set_stage_attrs(
    stage = "soma_identification",
    attr_dict = soma_products,
)

print(products)

{
  'decimation':
  {
    'decimation_parameters':
    {
      'decimation_ratio':0.25,
    },
    'segment_id':864691137198252609,
  },
  'soma_identification':
  {
    'glia_meshes':[],
    'nuclei_meshes':[<trimesh.Trimesh(vertices.shape=(15681, 3), faces.shape=(34028, 3))>, <trimesh.Trimesh(vertices.shape=(1677, 3), faces.shape=(3886, 3))>, <trimesh.Trimesh(vertices.shape=(907, 3), faces.shape=(2044, 3))>, <trimesh.Trimesh(vertices.shape=(600, 3), faces.shape=(1349, 3))>, <trimesh.Trimesh(vertices.shape=(598, 3), faces.shape=(1328, 3))>, <trimesh.Trimesh(vertices.shape=(6751, 3), faces.shape=(15696, 3))>, <trimesh.Trimesh(vertices.shape=(1504, 3), faces.shape=(2994, 3))>, <trimesh.Trimesh(vertices.shape=(898, 3), faces.shape=(1702, 3))>, <trimesh.Trimesh(vertices.shape=(511, 3), faces.shape=(952, 3))>],
    'soma_extraction_parameters':
    {
    },
    'soma_meshes':[<trimesh.Trimesh(vertices.shape=(16645, 3), faces.shape=(33075, 3))>],
    'soma_run_time':225.38343620300293,
    

# Step 2b: Saving off pipeline products

In [23]:
from datasci_tools import system_utils as su
su.save_object(
    products,"products_up_to_soma_stage"
)

Saved object at /NEURD/Applications/Tutorials/VDI_microns/products_up_to_soma_stage.pkl
File size is 19.018161 MB
Cleared mesh: <trimesh.Trimesh(vertices.shape=(15681, 3), faces.shape=(34028, 3))>:140033815377568
Cleared mesh: <trimesh.Trimesh(vertices.shape=(1677, 3), faces.shape=(3886, 3))>:140033815298160
Cleared mesh: <trimesh.Trimesh(vertices.shape=(907, 3), faces.shape=(2044, 3))>:140033806814464
Cleared mesh: <trimesh.Trimesh(vertices.shape=(600, 3), faces.shape=(1349, 3))>:140033815376944
Cleared mesh: <trimesh.Trimesh(vertices.shape=(598, 3), faces.shape=(1328, 3))>:140033815379392
Cleared mesh: <trimesh.Trimesh(vertices.shape=(6751, 3), faces.shape=(15696, 3))>:140033456237248
Cleared mesh: <trimesh.Trimesh(vertices.shape=(1504, 3), faces.shape=(2994, 3))>:140033456240144
Cleared mesh: <trimesh.Trimesh(vertices.shape=(898, 3), faces.shape=(1702, 3))>:140033458037088
Cleared mesh: <trimesh.Trimesh(vertices.shape=(511, 3), faces.shape=(952, 3))>:140033458039920
Cleared mesh: <t

# Step 3: Decomposition

In [ ]:
from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh,
    segment_id = segment_id, # don't need this explicitely if segment_id is already in products
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
)

neuron_obj

--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
limb_remove_mesh_interior_face_threshold = 0
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(16645, 3), faces.shape=(33075, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(16645, 3), faces.shape=(33075, 3))>]
soma_mesh_list_centers = [array([793840.75170922, 481099.37928507, 980586.79360769])]
Getting Glia and Nuclei Pieces Subtracted Away 3.4734129905700684
 Splitting mesh after soma cancellation 9.202287435531616
# of split_meshes = 25
 Containing Mesh Indices 0.7883341312408447
containing_mesh_indices = {0: 0}
 non_soma_touching_meshes 0.0003299713134765625
There were 24 pieces found after size threshold
 Finding inside pieces and non_soma_touching meshes 2.714883327484131
soma_containing_meshes = {0: [0]}

-----Before combining multiple mesh pieces-----
soma_containing_meshes = {0: [0]}

---

In [ ]:
nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)

# Calculate statistics

In [ ]:
decomp_products = neuron_obj.calculate_decomposition_products(
    store_in_obj = True,
)

decomp_products

In [ ]:
print(neuron_obj.pipeline_products)

# Saving off neuron

In [ ]:
vdi.save_neuron_obj(
    neuron_obj,
    verbose = True
)

# Reloading the Neuron

In [ ]:
neuron_obj_rec = vdi.load_neuron_obj(
    segment_id = segment_id,
    #mesh_decimated = mesh_decimated
)

In [ ]:
print(neuron_obj_rec.pipeline_products)

# Step 4A: Multi Soma Split Suggestions

In [ ]:
from neurd import neuron_visualizations as nviz
nviz.plot_soma_limb_concept_network(
    neuron_obj
)

In [ ]:
multi_soma_split_parameters = dict()

In [ ]:
_ = neuron_obj.calculate_multi_soma_split_suggestions(
    plot = True,
    store_in_obj = True,
    **multi_soma_split_parameters
)


In [ ]:
neuron_obj.pipeline_products.multi_soma_split_suggestions.multi_soma_split_parameters = multi_soma_split_parameters

# Step 4B: Multi Soma Split Execution

In [ ]:
neuron_list = neuron_obj.multi_soma_split_execution(
    verbose = False,
)

In [ ]:
n1 = neuron_list[0]
print(n1.pipeline_products)

# Step 5: Cell Typing

In [ ]:
from neurd import neuron_pipeline_utils as npu

In [ ]:
neuron_obj_axon = npu.cell_type_ax_dendr_stage(
    n1,
    mesh_decimated = mesh_decimated,
    plot_axon = False,
)

In [ ]:
nviz.plot_axon(
    neuron_obj_axon
)

# Autoproofreading Stage

In [ ]:
neuron_obj_proof = npu.auto_proof_stage(
    neuron_obj_axon,
    mesh_decimated = mesh_decimated,
    calculate_after_proof_stats = False,
)

In [ ]:
_ = npu.after_auto_proof_stats(
    neuron_obj_proof,
    store_in_obj = True,
)

# Saving off the autoproofread neuron

In [ ]:
vdi.save_neuron_obj_auto_proof(
    neuron_obj_proof,
)

# Plotting the final neuron

In [ ]:
from neurd import synapse_utils as syu

In [ ]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    neuron_obj_axon.mesh,
    meshes = [neuron_obj_proof.mesh],
    meshes_colors="red"
)

In [ ]:
syu.plot_synapses(neuron_obj_proof)

In [ ]:
syu.plot_synapses_error_from_neuron_obj(neuron_obj_proof)    

In [ ]:
syu.plot_synapses_valid_from_neuron_obj(neuron_obj_proof)  

In [ ]:
nviz.plot_compartments(neuron_obj_proof)